In [2]:
#importing libraries
import numpy as np 
import pandas as pd

In [3]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [4]:
X_full = pd.read_csv('train.csv',index_col="id")
X_test_full = pd.read_csv('test.csv', index_col='id')

# Remove rows with missing target, separate target from predictors
X_full.dropna(axis=0, subset=['sales'], inplace=True)
y = X_full.sales
X_full.drop(['sales'], axis=1, inplace=True)

#validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_full, y, 
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=0)


# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train_full.columns if
                    X_train_full[cname].nunique() < 20 and 
                    X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if 
                X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

In [5]:
X_train.head()

,store_nbr,onpromotion
id,,
1757797,3,0
820517,31,0
1464604,52,0
1736105,21,3
261037,33,0


In [6]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [7]:
model = RandomForestRegressor(n_estimators=50, random_state=0)

# Bundle preprocessing and modeling code in a pipeline
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])

# Preprocessing of training data, fit model 
clf.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = clf.predict(X_valid)

print('MAE:', mean_absolute_error(y_valid, preds))

MAE: 370.0858799896322


In [8]:
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

# Preprocessing of training data, fit model 
my_pipeline.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

# Evaluate the model
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)

MAE: 370.0858799896322


In [9]:
preds_test = my_pipeline.predict(X_test)

In [10]:
output = pd.DataFrame({'Id': X_test.index,
                       'sales': preds_test})
output.to_csv('submission.csv', index=False)

In [13]:
output



,Id,sales
0,3000888,125.788906
1,3000889,125.788906
2,3000890,429.143509
3,3000891,1358.255064
4,3000892,125.788906
...,...,...
28507,3029395,609.278250
28508,3029396,216.747951
28509,3029397,609.278250
28510,3029398,1408.764425
